# Stance and Sentiment Scoring with LLaMa-2

This notebook finetunes base LLaMa-2 7B to perform stance and sentiment analysis. In particular, the model will perform the following task:

* Given an excerpt of a news article as input as a prompt
* Answer
    1. What is the sentiment of the excerpt [Negative / Neutral / Positive]?
    2. What is the sentiment score assignable to this excerpt between -1.0 and 1.0?
    3. What is the reason behind the sentiment classification and score? Mention linguistic characteristics from the excerpt.
    4. What is the stance of the excerpt [Against-country / Impartial / Pro-country]?
    5. What is the stance score assignable to this excerpt between -1.0 and 1.0?
    6. What is the reason behind the stance classification and score? Mention linguistic characteristics from the excerpt.

## Setup

Installing dependencies and connecting huggingface through access token.

In [1]:
!pip install autotrain-advanced
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 102.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 

In [2]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [3]:
!autotrain llm \
--train \
--project_name "llama2-sentiment-and-stance" \
--model "TinyPixel/Llama-2-7B-bf16-sharded" \
--data_path /content/drive/MyDrive/News\ Data/train \
--text_column text \
--use_peft \
--use_int4 \
--learning_rate 2e-4 \
--train_batch_size 4 \
--num_train_epochs 5 \
--trainer sft \
--model_max_length 1024 \
--block_size 512 > training.log &

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='/content/drive/MyDrive/News Data/train', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model='TinyPixel/Llama-2-7B-bf16-sharded', model_ref=None, learning_rate=0.0002, num_train_epochs=5, train_batch_size=4, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=512, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='llama2-sentiment-and-stance', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=False, use_int8=False, model_max_length=1024, repo_id=None, use_int4=True, trainer='sft', target_modules=None, merge_adapter=False, token=None, backend='default', username=Non

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# copy it there
!cp -r /content/llama2-sentiment-and-stance /content/drive/MyDrive/model_dir

In [7]:
!pip install transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn import DataParallel

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/content/llama2-sentiment-and-stance")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/llama2-sentiment-and-stance")

In [1]:
input_content = """As a neutral news analyst, assess the sentiment and stance of the news article excerpt and assign a score between -1.0 (completely negative/against-canada) and 1.0 (completely positive/pro-canada) for both sentiment and stance. Provide a single short sentence to justify your scores, drawing on the article's language, tone, and presentation to support your analysis.

Article Excerpt:
- Title: "Calgary police say a 36-year-old man is facing a total of eight charges in a hate-motivated crime. Police say protesters disrupted a Reading with Royalty event early last month at the Seton Library in the citys southeast. The family-friendly storytimes at public libraries are led by local drag queens or kings, and children are invited to dress in their best outfit, cape or crown. Police say behaviour of protesters outside these events has been getting worse in recent weeks. Officers were called around 11 a.m. on Feb. 25 for reports that several people had aggressively entered a library classroom, shouting homophobic and transphobic slurs at the children and parents in attendance. Derek Scott Reimer has been arrested and charged with causing a disturbance and mischief. Reimer was due to make a court appearance Friday morning. Police say hate-motivated crimes are those where an offender was motivated by bias, prejudice or hate based on personal characteristics of the victim. 1:44 Calgary sees more protests against LGBTQ2+ community Hate motivation in a crime can be considered by the courts if a person is found guilty of the connected crime and can add to the sentencing. CPS Supt. Joe Brar told Global News police conducted a criminal investigation into all aspects of the incident and determined the charges, but no complaint was made against the people who removed Reimer from the Seton library event, as seen on videos in social media. Calgary bylaw officers also charged Reimer with six counts of harassment under the citys public behaviour bylaw, charges issued on a long-form summons. For each charge, Reimer could face up to $10,000 in fines and if payment defaults, six months imprisonment. Amendments to the citys bylaw in June 2022 prohibited harassing anyone for their age, race, sexual orientation, gender, religion, or gender identity or expression. These amendments were created as a clear message that Calgary is a place everyone should feel welcome and safe, and free from harassment, Ryan Pleckaitis, the citys chief of community standards, said in a statement. With more protests and counter-protests planned for city hall this weekend and into the early week, the city reminded citizens that laws and bylaws apply to all sides of any protest equally, and that police and bylaw officers will continue to ensure other incidents of harassment are monitored, investigated and enforced. Escalations in anti-LGBTQ2S+protests Brar said over the past few months, 14 events like reading with royalty, drag brunches and shows at the Chinook Blast winter festival, have been targeted by anti-LGBTQ2S+ protesters. And over the weeks, their behaviour has gotten worse, he said. He said police have had to change their response to that worsening behaviour. Its escalated from what would be acceptable or expected in a protest or a demonstration to what is obviously intended to be harassment and intimidation and breach of the peace and all those things, under the guise of a protest, Brar said. Thats having a serious impact against peoples sense of physical and psychological security and also having huge impacts on the communities that the protests are directed at, in addition to the geographical locations. King Neptune, one of the drag performers at the Seton Library event over the weekend, said the anti-LGBTQ2S+ hate is especially loud in Calgary. Unfortunately, Calgary itself has become a bit of a hub for this sort of hatred. Youll see these events pop up in Edmonton and Saskatchewan and Vancouver and therell be the occasional protest, but weve seen kind of a sustained campaign against these LGBTQ, especially all ages, events or youth events really have a heavy target. So this is very much a Calgary issue, Neptune told Global News. On Friday, the Calgary Public Library (CPL) announced it was postponing its Reading with Royalty event at the Southwood Library scheduled for Saturday. CPL said it will reschedule the event when it can ensure the safety and fun environment for kids and parents, as part of its commitment to the partnership with Calgary Pride. The right to protest is fundamental to our democracy, but what happened last weekend inside Seton Library was not a protest. It was an organized, targeted and intimidating disruption of a program in a space where small children were present, CPL said in a statement. CPL has partnered with Pride to host Reading with Royalty events for more than five years, describing it as a popular, family-friendly event that celebrates inclusion and imagination through stories and songs. All performers and staff complete training to ensure that events are kid-friendly and entertaining. The suggestion that it is anything else is categorically untrue and rooted in misinformation and intolerance, CPL added. Tweet This Click to share quote on Twitter: "All performers and staff complete training to ensure that events are kid-friendly and entertaining. The suggestion that it is anything else is categorically untrue and rooted in misinformation and intolerance," CPL added. Police said Reimers are the first hate crime-related charges coming out of the recent spate of protests, but Neptune has concerns that those spewing anti-LGBTQ2S+ rhetoric might see the event postponements as a sort of victory. Unfortunately, things like this can embolden them to feel like theyre still getting more news coverage and sometimes even negative news is news and its publicity. Were going to take (Reimers charges as a) win, but we know that this doesnt dissuade them from continuing this hate campaign that they have. The police superintendent reiterated that Calgary police respect Charter rights to freedom of expression and peaceful protest, and are often in attendance to make sure thats done in a safe and lawful manner. We will be taking enforcement action to make sure that we create a safe environment for everybody, including those people, those individuals or those groups that are being protested against. And its very important to know that these protests, although being done under the freedom of expression and your democratic rights to protest, are having an impact on a very specific community, Brar added. Dressed in a pantsuit, shirt and bow tie, Neptune said the performers at Reading with Royalty events are trying to bring messages of positivity and inclusivity. I know what Im doing is perfectly lovely and OK, and Im spreading joy, Im spreading love as a performer, Neptune said. So when I go to these events, I am not often feeling dread about (the protests). Its kind of that same feeling of Its 2023 and were still doing this. Police interventions critical Kristopher Wells, an associate professor and Canada Research Chair for the public understanding of sexual and gender minority youth at MacEwan University, said Canadas laws are clear about hate speech and hate incidents, and should be pursued by police in incidents like the one at the Seton Library and the Canyon Meadows pool two weeks prior. Its critical, because these kinds of attacks on the community, they dont just attack an event or an individual, but theyre designed to instil fear and terror into an entire community, to tell them that you dont belong here, that if youre going to be visible and vocal, there will be consequences for that, Wells told 630 CHED. He also said the federal government should strengthen the Criminal Code to make it easier to prosecute hate crimes and hate incidents. They get masked under this banner of religious freedom or freedom of speech when in fact, what it is is its hate speech and it needs to be treated as such. Tweet This Click to share quote on Twitter: "They get masked under this banner of religious freedom or freedom of speech when in fact, what it is is it's hate speech and it needs to be treated as such." Wells warned about the creep of anti-LGBTQ2S+ ideology from the far right, despite pan-continental and historic evidence of those identities existing. It seems that the more that sometimes were moving forward with human rights and LGBTQ equality here in Canada, were seeing steps backwards in other parts of the world, including just south of our border in the United States, with many of these anti-trans health-care laws that are being passed, which quite frankly, are putting vulnerable peoples lives at great risk, he said. Whats happening in the United States in particular is caught up in what were seeing happen to reproductive rights and, womens rights, access to abortion  this is all linked together under these notions of bodily autonomy and ultimately who should have the right to make important decisions about their own bodies and their own lives. Marni Panas, a Canadian certified inclusion professional and trans advocate, said the rhetoric of anti-LGBTQ2S+ protests that target drag shows or events like Reading with Royalty and anti-trans legislation introduced in some of the United States, is intended to eliminate transgender people from society. Panas, a transgender woman, was invited to speak at Discovery House on Wednesday as part of International Womens Day celebrations. She said she has received a number of threats because of her upcoming speech at a shelter designed to support woman and children fleeing domestic violence. Panas and organizers have had to engage police and security to ensure the safety of everyone in attendance. These are the things I have to think about, but have never had to think about before, Panas told 630 CHED. Panas helped update Canadas Human Rights Act and Criminal Code to add gender identity and gender expression under the definition of hate crime, a necessary protection for a vulnerable community. But increased hateful rhetoric concerns Panas. At some point, somebody will die if we continue to allow this hateful rhetoric to occur in these public spaces and online, and so these protections need to be enforced and police services need to do much more to support these vulnerable communities, she said. with files from The Canadian Press"
- Content: "Calgary man faces police, bylaw charges following hate-motivated library protest"

Output format:
1. Sentiment: [Positive/Neutral/Negative]
    * Score: [Your Score]
    * Reason: [Your Reason]
2. Stance: [Pro-canada/Impartial/Against-canada]
    * Score: [Your Score]
    * Reason: [Your Reason]
"""